In [1]:
#!/usr/declass/env/bin/python
# -*- coding: utf-8 -*-

import os
import sys
import pymysql
import ConfigParser
import pandas as pd

# get config to connect to local dbs
config = ConfigParser.RawConfigParser()
config.readfp(open(r'../mylogin.cnf'))
db_user = config.get('client', 'user')
db_pass = config.get('client', 'password')

frus_db = pymysql.connect(host='history-lab.org', user=db_user, passwd=db_pass, db="declassification_frus", charset='utf8')

persons = pd.read_sql("SELECT * FROM persons;", con=frus_db)

In [2]:
persons.head()

,id,name,birth_year,death_year,description
0,100001,"Aaron, David Laurence",1938.0,NaN,President’s Deputy Assistant for National Secu...
1,100002,"Aaron, Harold R.",NaN,NaN,"Major General, USA; Assistant Chief of Staff, ..."
2,100003,"Abaydi, Haamid al-",NaN,NaN,Libyan Defense Minister until September 1969
3,100004,"Abbas, Ferhat",NaN,NaN,Prime Minister of the Provisional Government o...
4,100005,"Abbas, M. M.",NaN,NaN,"Director General for Americas and Europe, Mini..."


In [3]:
persons.to_csv('declass_persons.csv', encoding='utf8')

In [4]:
scientists = pd.read_sql("SELECT * FROM persons WHERE description LIKE '%scien%';", con=frus_db)

In [5]:
scientists.head()

,id,name,birth_year,death_year,description
0,100529,"Arbatov, Georgi A.",NaN,NaN,"Director, Soviet Institute for U.S.A. and Cana..."
1,100538,"Arculus, Ronald",NaN,NaN,"Head of the Science and Technology Department,..."
2,100591,"Arneson, R. Gordon",NaN,NaN,"Deputy Director of Intelligence and Research, ..."
3,100660,"Atkinson, Richard C.",NaN,NaN,Director of the National Science Foundation; A...
4,100688,"Averch, Harvey A.",NaN,NaN,economist; Assistant Director for Science Educ...


In [6]:
scientists['description'] = scientists['description'].str.lower()
scientists['name'] = scientists['name'].str.lower()
scientists = scientists[['id', 'name', 'description']]

In [7]:
soviet_scientists = scientists[scientists['description'].str.contains("soviet")]
soviet_scientists

,id,name,description
0,100529,"arbatov, georgi a.","director, soviet institute for u.s.a. and cana..."
15,101381,"blagonravov, anatoliy arkadievich",academician and chairman of the commission on ...
90,106987,"keldysh, mstislav v.","president, soviet academy of sciences; preside..."
91,107019,"kennan, george frost","historian; director, policy planning staff and..."
95,107227,"kirillin, vladimir a.",chairman of the state committee of the ussr co...
114,108431,"malone, thomas","representative, national academy of sciences, ..."
124,109409,"morozov, platon dmitrievich",soviet deputy permanent representative to the ...
159,111819,"sakharov, andrei dmitrievich",soviet dissident; soviet dissident; nobel peac...
172,112330,"shchukin, aleksandr n.",member of the soviet salt delegation; soviet s...


In [8]:
sovsci_ids = str(list(soviet_scientists['id'].astype(int).astype(str).values)).replace("[", "(").replace("]", ")")
sovsci_ids

"('100529', '101381', '106987', '107019', '107227', '108431', '109409', '111819', '112330')"

In [9]:
sovsci_person_doc = pd.read_sql("SELECT doc_id FROM person_doc WHERE person_id IN" + sovsci_ids + ";", con=frus_db)

In [10]:
sovsci_doc_ids = str(list(sovsci_person_doc['doc_id'].astype(str).values)).replace("[", "(").replace("]", ")")

In [11]:
sovsci_docs = pd.read_sql("SELECT * FROM docs WHERE id IN" + sovsci_doc_ids + ";", con=frus_db)

In [12]:
sovsci_docs.head()

,id,volume_id,chapt_title,title,title_docview,subject,date,location,p_from,p_to,source,classification,body,raw_body,collection
0,frus1943CairoTehrand303,frus1943CairoTehran,6. Conference Documents and Supplementary Papers:,The Secretary of\n ...,The Secretary of\n ...,None,1943-11-24,Washington,None,None,None,None,"<div collection=""frus"">\n<a href=""#frus1943Cai...",\n \nsecret\nFor the President from Secretary ...,frus
1,frus1943CairoTehrand466,frus1943CairoTehran,13. Conference Documents and Supplementary Pap...,The Secretary of\n ...,The Secretary of\n ...,None,1943-12-02,Washington,None,None,None,None,"<div collection=""frus"">\n<a href=""#frus1943Cai...",\n \nsecret\nFor the President From Secretary ...,frus
2,frus1943CairoTehrand480,frus1943CairoTehran,13. Conference Documents and Supplementary Pap...,The Secretary of\n ...,The Secretary of\n ...,None,1943-12-04,Washington,None,None,None,None,"<div collection=""frus"">\n<a href=""#frus1943Cai...",\n\nMemorandum for the President From the Secr...,frus
3,frus1945-50Inteld154,frus1945-50Intel,Souers' Tenure as Director of Central\n ...,154. Memorandum From the Director of Central I...,154. Memorandum From the Director of Central I...,Progress Report on the Central Intelligence Group,1946-06-07,Washington,Souers,None,Source: Central Intelligence Agency Historical...,Top Secret,"<div collection=""frus""> <p>\n1. Establishment\...",\n1. Establishment\n\nThe Central Intelligenc...,frus
4,frus1945-50Inteld256,frus1945-50Intel,Psychological and Political Warfare,256. Department of State Briefing Memorandum,256. Department of State Briefing Memorandum<a...,Report to National Security Council on Coordin...,1947-12-17,Washington,None,None,Source: National Archives and Records Administ...,Top Secret,"<div collection=""frus""> <p>COORDINATION OF FOR...",COORDINATION OF FOREIGN INFORMATION MEASURES ...,frus


In [13]:
sovsci_term_doc = pd.read_sql("SELECT * FROM term_doc WHERE doc_id IN" + sovsci_doc_ids + ";", con=frus_db)

In [14]:
sovsci_term_doc.head()

,term_id,doc_id,term_count
0,t_CIG1,frus1945-50Inteld154,28
1,t_JANIS1,frus1945-50Inteld154,1
2,t_NIA1,frus1945-50Inteld154,4
3,t_JCS1,frus1945-50Inteld256,1
4,t_NSC1,frus1945-50Inteld256,10


In [15]:
sovsci_term_ids = str(list(sovsci_term_doc['term_id'].astype(str).values)).replace("[", "(").replace("]", ")")

In [16]:
sovsci_terms = pd.read_sql("SELECT * FROM terms WHERE id IN" + sovsci_term_ids + ";", con=frus_db)

In [17]:
sovsci_terms.head()

,id,acronym,def
0,t_A1,A,"Office of the Assistant (after October 3, 1949..."
1,t_ABM1,ABM,anti-ballistic missile system
2,t_ACC1,ACC,"Administrative Committee on Coordination, UN E..."
3,t_ACDA1,ACDA,United States Arms Control and Disarmament Agency
4,t_AEC1,AEC,"Atomic Energy Commission AID, Agency for Inter..."


In [18]:
sakharov_person_docs = sovsci_person_doc = pd.read_sql("SELECT doc_id FROM person_doc WHERE person_id='111819';", con=frus_db)
sakharov_doc_ids = str(list(sakharov_person_docs['doc_id'].astype(str).values)).replace("[", "(").replace("]", ")")

In [19]:
sakharov_term_doc = pd.read_sql("SELECT * FROM term_doc WHERE doc_id IN" + sakharov_doc_ids + ";", con=frus_db)
sakharov_term_ids = str(list(sakharov_term_doc['term_id'].astype(str).values)).replace("[", "(").replace("]", ")")

In [20]:
sakharov_terms = pd.read_sql("SELECT * FROM terms WHERE id IN" + sakharov_term_ids + ";", con=frus_db)
sakharov_terms.head()

,id,acronym,def
0,t_AP_1,AP,Associated Press
1,t_ARA_1,ARA,"or ARA/LA, Bureau of Inter-American Affairs, D..."
2,t_ASNE_1,ASNE,American Society of Newspaper Editors
3,t_B1_1,B-1,"American long-range bomber\nBA, Bachelor of Ar..."
4,t_CBS_1,CBS,Columbia Broadcasting System


In [21]:
# sakharov_terms.to_csv('sakharov_terms.csv',encoding='utf8')

In [22]:
total_terms = pd.read_sql("SELECT * FROM terms;", con=frus_db)
# total_terms.to_csv('frus_terms.csv',encoding='utf8')

In [23]:
total_term_doc = pd.read_sql("SELECT * FROM term_doc;", con=frus_db)
# total_term_doc.to_csv('frus_term_doc.csv',encoding='utf8')